In [121]:
from main import *

In [122]:
league_name = 'rpk'  #'rpk', 'ifc', 'rbsc'
league_id = get_league_id(league_name)

In [123]:
gw_no = 3

In [124]:
# see gameweek general info
r = requests.get('https://fantasy.premierleague.com/api/'+'bootstrap-static/').json()

In [125]:
r.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [126]:
gw_info = r['events'][gw_no - 1]

In [127]:
gw_info # want this kind of info for our leagues especially chip plays, most_selected, most_captained

{'id': 3,
 'name': 'Gameweek 3',
 'deadline_time': '2024-08-31T10:00:00Z',
 'release_time': None,
 'average_entry_score': 64,
 'finished': True,
 'data_checked': True,
 'highest_scoring_entry': 430195,
 'deadline_time_epoch': 1725098400,
 'deadline_time_game_offset': 0,
 'highest_score': 130,
 'is_previous': True,
 'is_current': False,
 'is_next': False,
 'cup_leagues_created': True,
 'h2h_ko_matches_created': True,
 'ranked_count': 9912161,
 'chip_plays': [{'chip_name': 'bboost', 'num_played': 87334},
  {'chip_name': 'freehit', 'num_played': 140192},
  {'chip_name': 'wildcard', 'num_played': 656652},
  {'chip_name': '3xc', 'num_played': 186907}],
 'most_selected': 351,
 'most_transferred_in': 177,
 'top_element': 328,
 'top_element_info': {'id': 328, 'points': 17},
 'transfers_made': 23723836,
 'most_captained': 351,
 'most_vice_captained': 328}

# Post match

In [128]:
assert gw_info['finished']

In [129]:
points = get_gw_points(league_id, gw_no)
points

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank
0,5961532,Aekk72,Atthapon Parkart,3,93,0,93,1
1,3611665,TachaponR,Tachapon Ratsameedara,3,74,0,74,2
6,2910243,Morty FC,Pattapong Charoenchaipong,3,74,0,74,2
4,1576908,Chicks Casino CF,Tawiwut Charuwat,3,73,0,73,3
5,5377619,PTHOMAS FC,PHOOM T. YENBAMROONG,3,71,0,71,4
2,7348222,Sirawatd,sirawat dulyavit,3,68,0,68,5
3,4329910,proove.club,Natthawat Charoenkitmongkol,3,62,0,62,6
7,3607641,OnkaewmaneeN.,Nithiz Onkaewmanee,3,53,0,53,7


In [130]:
pnl = get_gw_weekly_pnl(points, league_name)
pnl

,manager_id,team_name,name,gw_no,points,transfers_cost,h2h_points,rank,pnl,is_unique_rank
0,5961532,Aekk72,Atthapon Parkart,3,93,0,93,1,NaN,False
1,3611665,TachaponR,Tachapon Ratsameedara,3,74,0,74,2,NaN,False
6,2910243,Morty FC,Pattapong Charoenchaipong,3,74,0,74,2,NaN,False
4,1576908,Chicks Casino CF,Tawiwut Charuwat,3,73,0,73,3,NaN,False
5,5377619,PTHOMAS FC,PHOOM T. YENBAMROONG,3,71,0,71,4,NaN,False
2,7348222,Sirawatd,sirawat dulyavit,3,68,0,68,5,NaN,False
3,4329910,proove.club,Natthawat Charoenkitmongkol,3,62,0,62,6,NaN,False
7,3607641,OnkaewmaneeN.,Nithiz Onkaewmanee,3,53,0,53,7,NaN,False


In [136]:
try:
    # rpk only equal rank pnl hardcode
    if not pnl.is_unique_rank[0]:
        raise ValueError("Please hard code PNL")
        # pnl['pnl'] = [300,150,150,0,0,-100,-200,-300] # gw3
        # assert sum(pnl['pnl'] == 0)
        # pnl
except AttributeError:
    # not rpk and no is_unique_rank column
    pass

ValueError: Please hard code PNL

In [132]:
# text output for reporting
print(f'GW {gw_no} H2H results\n')

print('Top scorer(s):')
for index, row in pnl.iterrows():
    # มันเรียงมาตั้งแต่ Logic pnl
    if row['pnl'] > 0:
        print(f"{row['team_name']} ({row['name']}) {row['points']} - {row['transfers_cost']} = {row['h2h_points']} Pts [{row['pnl']}]\n")
    else:
        break

print('\n\nBottom(s) of the week:')    
for index, row in pnl.iterrows():
    # มันเรียงมาตั้งแต่ Logic pnl
    if row['pnl'] < 0:
        print(f"{row['team_name']} ({row['name']}) {row['points']} - {row['transfers_cost']} = {row['h2h_points']} Pts & [{row['pnl']}]\n")


GW 3 H2H results

Top scorer(s):


Bottom(s) of the week:


In [120]:
# write data into csv
# Specify the location and file name
file_location = f'data/2024-2025/fact_gw_{league_name}.csv'

# Write the DataFrame to a CSV file
# pnl.to_csv(file_location, index=False)

# Append the DataFrame to the existing CSV file
pnl.to_csv(file_location, mode='a', index=False, header=False)

# Pre match
- Players data
- Managers data
- Managers picks
    - captain selection
    - most transfers
    - selected by %

## Player data

In [ ]:
# get latest player data, fun data to look at
players = get_dim_players()

In [ ]:
players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [ ]:
players.loc[players.id == 9, ['first_name', 'second_name']]

,first_name,second_name
8,Gabriel,Martinelli Silva


## Manager data

In [ ]:
managers = get_dim_managers(league_id)

In [ ]:
managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             55 non-null     int64  
 1   joined_time                    55 non-null     object 
 2   started_event                  55 non-null     int64  
 3   favourite_team                 52 non-null     float64
 4   player_first_name              55 non-null     object 
 5   player_last_name               55 non-null     object 
 6   player_region_id               55 non-null     int64  
 7   player_region_name             55 non-null     object 
 8   player_region_iso_code_short   55 non-null     object 
 9   player_region_iso_code_long    55 non-null     object 
 10  years_active                   55 non-null     int64  
 11  summary_overall_points         55 non-null     int64  
 12  summary_overall_rank           55 non-null     int64

## Manager picks

In [ ]:
gw_picks = get_gw_picks(league_id, gw_no)

In [ ]:
gw_picks

,element,position,multiplier,is_captain,is_vice_captain,manager_id
0,15,1,1,False,False,3481605
1,18,2,1,False,False,3481605
2,255,3,1,False,False,3481605
3,340,4,1,False,False,3481605
4,291,5,1,False,False,3481605
...,...,...,...,...,...,...
820,389,11,1,False,False,7104076
821,413,12,0,False,False,7104076
822,129,13,0,False,False,7104076
823,210,14,0,False,False,7104076


In [ ]:
# join with other data for description
gw_picks = pd.merge(gw_picks, players, left_on='element', right_on='id') # get player names
gw_picks = pd.merge(gw_picks, managers, left_on='manager_id', right_on='id') # get manager names

### Captains

In [ ]:
captains = gw_picks.loc[gw_picks.is_captain == True]


In [ ]:
# who captain who?
captains.loc[
    :,
    ['first_name', 'second_name', 'player_first_name', 'player_last_name', 'name']
    ].sort_values('first_name')

,first_name,second_name,player_first_name,player_last_name,name
454,Bukayo,Saka,Cobby,Leathers,FC Cobbra
49,Bukayo,Saka,Tor,Na Ranong,TNR30
667,Bukayo,Saka,sirawat,dulyavit,Sirawatd
156,Bukayo,Saka,Tawiwut,Charuwat,Chicks Casino CF
472,Cole,Palmer,Teddy,T,Ted in the Box
400,Cole,Palmer,Sira,H,Pending Moderation
533,Cole,Palmer,Pimadej,Siwapornpitak,Fame’s Squad
276,Cole,Palmer,Nick,Thanapoomikul,NT
640,Cole,Palmer,Guitar,Raktakanit,Guitar29
233,Cole,Palmer,Sirichai,Jirapongphan,Timmy_AC


In [ ]:
# summary
captained_by = captains.groupby(['element', 'first_name', 'second_name']).size().sort_values(ascending=False).reset_index(name='managers_captained_by')
captained_by['percent_captained_by'] = round(captained_by.managers_captained_by / managers.shape[0] * 100, 2)

In [ ]:
captained_by

,element,first_name,second_name,managers_captained_by,percent_captained_by
0,351,Erling,Haaland,33,60.00
1,182,Cole,Palmer,13,23.64
2,17,Bukayo,Saka,4,7.27
3,328,Mohamed,Salah,4,7.27
4,345,Kevin,De Bruyne,1,1.82


### most selected

In [ ]:
selected_by = gw_picks[gw_picks.multiplier != 0].groupby(['element', 'first_name', 'second_name']).size().sort_values(ascending=False).reset_index(name='managers_selected_by')

In [ ]:
selected_by['percent_selected_by'] = round(selected_by.managers_selected_by / managers.shape[0] * 100, 2)

In [ ]:
selected_by.head(10)

,element,first_name,second_name,managers_selected_by,percent_selected_by
0,351,Erling,Haaland,43,78.18
1,401,Alexander,Isak,34,61.82
2,17,Bukayo,Saka,28,50.91
3,19,Emile,Smith Rowe,26,47.27
4,350,Joško,Gvardiol,24,43.64
5,328,Mohamed,Salah,23,41.82
6,182,Cole,Palmer,22,40.00
7,317,Diogo,Teixeira da Silva,21,38.18
8,311,Trent,Alexander-Arnold,19,34.55
9,255,Antonee,Robinson,17,30.91


In [ ]:
# have both Salah and Haaland?
gw_picks[gw_picks.element.isin([351,328])].groupby(['manager_id', 'player_first_name', 'player_last_name', 'name']).size().sort_values(ascending=False)

manager_id  player_first_name  player_last_name  name                
6550939     Pow                …..               pow                     2
2891449     Akin               Suriyabhivadh     Akin FC                 2
6031430     Pum                Lamsam            Slot into Diaz          2
4457564     Chuan              Krub              Chuan United ⚽️         2
2271302     Pimadej            Siwapornpitak     Fame’s Squad            2
3481605     Napin              Surakkhaka        SNAPIN                  2
1831372     Wanwiwat           Thongproh         Customsanova FC         2
5186410     Gampanath          Udomsorayuth      ganpgampLFC             2
1203040     Samote             Viranuvatti       Pun FC                  2
969423      Fung               Niamkongkit       Team 14                 2
4473266     Tae                Uahwatanasakul    Drogba                  2
2890749     BooM               Suriyabhivadh     69 United               2
4778474     Sira              

In [ ]:
# Salah and Haaland
for manager_id in managers.id:
    gw_pick = get_manager_picks(manager_id, gw_no)['picks']
    pick_index = 0
    has_h = False
    has_s = False
    for pick in gw_pick:
        if pick['element'] == 351:
            has_h = True
        elif pick['element'] == 328:
            has_s = True
    if has_h and has_s:
        print(manager_id, has_h, has_s)

5186410 True True
3481605 True True
2271302 True True
4457564 True True
6550939 True True
1203040 True True
1831372 True True
2891449 True True
4473266 True True
2890749 True True
969423 True True
6031430 True True


In [ ]:
# Salah and Haaland
h_s = []
h_nos = []
noh_s = []
noh_nos = []
for manager_id in managers.id:
    gw_pick = get_manager_picks(manager_id, gw_no)['picks']
    pick_index = 0
    has_h = False
    has_s = False
    for pick in gw_pick:
        if pick['element'] == 351:
            has_h = True
        elif pick['element'] == 328:
            has_s = True
    if has_h and has_s:
        h_s.append(manager_id)
    elif has_h and not has_s:
        h_nos.append(manager_id)
    elif not has_h and has_s:
        noh_s.append(manager_id)
    else:
        noh_nos.append(manager_id)
print(len(h_s),len(h_nos),len(noh_s),len(noh_nos))

12 27 10 5


In [ ]:
noh_nos

[7307751, 7104076, 4944128, 4087164, 2642311]

In [ ]:
# rpk only equal rank pnl hardcode
# points['pnl'] = []

### Chips (work in progress)

In [ ]:
for manager_id in managers.id:
    print(manager_id)
    print(type(manager_id))

8264377
<class 'int'>
8151266
<class 'int'>
1576908
<class 'int'>
1183765
<class 'int'>
2893557
<class 'int'>
6949233
<class 'int'>
5908058
<class 'int'>
3961683
<class 'int'>
6301628
<class 'int'>
779014
<class 'int'>
3552329
<class 'int'>
2968038
<class 'int'>
6037486
<class 'int'>
5337932
<class 'int'>
5224099
<class 'int'>
3613439
<class 'int'>
8419308
<class 'int'>
6669005
<class 'int'>
6597853
<class 'int'>


In [ ]:
for manager_id in managers.id:
    active_chip = get_manager_picks(manager_id, gw_no)['active_chip']
    if active_chip:
        print(active_chip)
        print(managers.loc[managers.id == manager_id, ['player_first_name', 'player_last_name', 'name']])

wildcard
  player_first_name player_last_name      name
3          Chotiwit   Jiratananuwong  Meatball
wildcard
  player_first_name player_last_name      name
5          Peerarat   Laolugsanalerd  peerarat


### transfers

In [ ]:
# TODO

# End of season

In [ ]:
# df = pd.read_csv('./data/rbsc/fact_gw_rbsc.csv')
# for gw_no in range(37,38+1):
#     df = pd.concat([df, get_gw_weekly_pnl(get_gw_points(league_id, gw_no), league_name)], ignore_index=True)
#     gw_no += 1